In [2]:
import numpy as np
import pandas as pd
import datetime as dt
import random
import yfinance as yf
import os
from os import listdir
from os.path import isfile, join

PATH = 'E:\\Projects\\Finoobs\\Portfolio Optimization\\stocks\\'
risk_free_rate = 0.0125

def get_df_from_csv(ticker):
    try:
        df = pd.read_csv(PATH + ticker + '.csv', index_col='Date', parse_dates=True)
    except FileNotFoundError:
        pass
    else:
        return df

def merge_df_by_column_name(col_name, sdate, edate, *tickers):
    mult_df = pd.DataFrame()
    sdate = pd.to_datetime(sdate)
    edate = pd.to_datetime(edate)

    for x in tickers:
        df = get_df_from_csv(x)
        df.index = pd.to_datetime(df.index)
        mask = (df.index >= sdate) & (df.index <= edate)
        mult_df[x] = df.loc[mask][col_name]
        
    return mult_df

def get_random_files(folder_path, num_files=None):
    file_list = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]
    
    if num_files is None or num_files > len(file_list):
        num_files = len(file_list)
    
    random_files = random.sample(file_list, num_files)
    return [file[:-4] for file in random_files]

def optimize_portfolio(investment_amount):

    num_files_to_select = int(input("Enter the number of stocks: "))
    
    port_list = get_random_files(PATH, num_files_to_select)
    

    end_date = dt.datetime.now().strftime('%Y-%m-%d')
    mult_df = merge_df_by_column_name('Adj Close', '2019-01-01', end_date, *port_list)
    mult_df = mult_df.apply(pd.to_numeric, errors='coerce')

    returns = np.log(mult_df / mult_df.shift(1))
    
    p_ret = []
    p_vol = []
    p_SR = []
    p_wt = []

    port_list = [ticker + ".NS" for ticker in port_list]
    # print(port_list)
    
    for _ in range(10000):
        weights = np.random.random(num_files_to_select)
        weights /= np.sum(weights)
        
        ret = np.sum(weights * returns.mean()) * 252
        vol = np.sqrt(np.dot(weights.T, np.dot(returns.cov() * 252, weights)))
        sr = (ret - risk_free_rate) / vol
        
        p_ret.append(ret)
        p_vol.append(vol)
        p_SR.append(sr)
        p_wt.append(weights)

    p_ret = np.array(p_ret)
    p_vol = np.array(p_vol)
    p_SR = np.array(p_SR)
    p_wt = np.array(p_wt)

    optimal_idx = np.argmax(p_SR)
    optimal_weights = p_wt[optimal_idx]

    # Get current prices
    current_prices = {}
    for stock in port_list:
        # stock = stock+'.NS'
        ticker = yf.Ticker(stock)
        try:
            current_prices[stock] = ticker.info.get('currentPrice', np.nan)
            
        except KeyError:
            print(f"Price data for {stock} is not available.")
            continue
    print(current_prices)

    # Calculate number of shares and investment amount for each stock
    total_investment = 0
    recommendations = []
    for i, stock in enumerate(port_list):
        
        amount = investment_amount * optimal_weights[i]
        shares = int(amount / current_prices[stock])
        actual_investment = shares * current_prices[stock]
        total_investment += actual_investment
        recommendations.append({
            'stock': stock,
            'weight': optimal_weights[i],
            'shares': shares,
            'investment': actual_investment
        })

    return recommendations, p_ret[optimal_idx], p_vol[optimal_idx], total_investment

def main():
    investment_amount = float(input("Enter the total investment amount: "))
    recommendations, expected_return, volatility, total_invested = optimize_portfolio(investment_amount)

    print("\nInvestment Recommendations:")
    for rec in recommendations:
        print(f"Stock: {rec['stock']}")
        print(f"  Weight: {rec['weight']:.2%}")
        print(f"  Shares: {rec['shares']}")
        print(f"  Investment: ${rec['investment']:.2f}")
        print()

    print(f"Total Invested: ${total_invested:.2f}")
    print(f"Expected Annual Return: {expected_return:.2%}")
    print(f"Expected Volatility: {volatility:.2%}")
    print(f"Sharpe Ratio: {(expected_return - risk_free_rate) / volatility:.2f}")

if __name__ == "__main__":
    main()

['IDEAFORGE.NS', 'URAVI.NS', 'GUFICBIO.NS', 'EMCURE.NS', 'IXIGO.NS', 'SHOPERSTOP.NS', 'MARSHALL.NS', 'CONSOFINVT.NS', 'IMPEXFERRO.NS', 'SHILPAMED.NS', 'IVC.NS']
{'IDEAFORGE.NS': 575.5, 'URAVI.NS': 487.0, 'GUFICBIO.NS': 439.75, 'EMCURE.NS': 1440.8, 'IXIGO.NS': 149.48, 'SHOPERSTOP.NS': 682.15, 'MARSHALL.NS': 30.66, 'CONSOFINVT.NS': 206.85, 'IMPEXFERRO.NS': 3.13, 'SHILPAMED.NS': 842.75, 'IVC.NS': 12.29}

Investment Recommendations:
Stock: IDEAFORGE.NS
  Weight: 3.37%
  Shares: 70
  Investment: $40285.00

Stock: URAVI.NS
  Weight: 4.02%
  Shares: 98
  Investment: $47726.00

Stock: GUFICBIO.NS
  Weight: 11.42%
  Shares: 311
  Investment: $136762.25

Stock: EMCURE.NS
  Weight: 28.83%
  Shares: 240
  Investment: $345792.00

Stock: IXIGO.NS
  Weight: 6.46%
  Shares: 518
  Investment: $77430.64

Stock: SHOPERSTOP.NS
  Weight: 3.23%
  Shares: 56
  Investment: $38200.40

Stock: MARSHALL.NS
  Weight: 0.68%
  Shares: 267
  Investment: $8186.22

Stock: CONSOFINVT.NS
  Weight: 3.63%
  Shares: 210
  I